# Purpose

dspy を試す。

Reference:

- [DSPy](https://dspy.ai)
- [DSPyで始めるプロンプト最適化](https://zenn.dev/kurushi/articles/b26baa1e41c194) (in Japanese)
  - これは古い（2024 年の記事）
- [LLM用宣言的プログラミング言語 DSPy](https://zenn.dev/cybernetics/articles/f879e10b53c2db) (in Japanese)
- [プロンプトエンジニアリングを終わらせるDSPy](https://zenn.dev/cybernetics/articles/39fb763aca746c) (in Japanese)

In [4]:
from getpass import getpass
import os
import dspy

In [2]:
dspy.__version__

'3.0.4'

## Setup LM

In [11]:
model_id: str = "gemini/gemini-3-flash-preview"
api_key_env: str = "GEMINI_API_KEY"

lm = dspy.LM(model_id, api_key=os.getenv(api_key_env))
if os.getenv(api_key_env) is None:
    lm = dspy.LM(model_id, api_key=getpass("Enter your Gemini API key:"))

dspy.configure(
    lm=lm, temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

## test 1

- [LLM用宣言的プログラミング言語 DSPy](https://zenn.dev/cybernetics/articles/f879e10b53c2db) (in Japanese)

In [12]:
task = "足し算をしてください。{num1} たす {num2}"

lm(task.format(num1="いち", num2="ご"))[0]

'いち たす ご は **ろく**（6）です。'

## test 2

- [プロンプトエンジニアリングを終わらせるDSPy](https://zenn.dev/cybernetics/articles/39fb763aca746c) (in Japanese)

### Before optimization

In [13]:
# ナルト口調変換用のシグネチャとモジュール定義
class NarutoSignature(dspy.Signature):
    polite_sentence = dspy.InputField(desc="です・ます調の落ち着いた一文")
    rationale = dspy.OutputField(desc="ナルトの喋りへ変換する際の推論過程")
    transformed = dspy.OutputField(desc="ナルト口調に変換した文。『でもさ、』以降に余計な一言を必ず添える")

class NarutoStyleChain(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.ChainOfThought(NarutoSignature)

    def forward(self, polite_sentence):
        return self.generator(polite_sentence=polite_sentence)

make_naruto_chain = NarutoStyleChain()

In [14]:
polite_sentence = "今日の会議では議論を丁寧にまとめます。"
easy_response = make_naruto_chain(polite_sentence=polite_sentence)

In [15]:
easy_response

Prediction(
    reasoning='この文章は「会議」という公的な場での「丁寧なまとめ」を宣言しています。ナルトであれば、難しい会議でもやる気を出しつつ、彼らしい直球で自信のある言い回し（「～だぜ」「～ばよ」）に変換するのが自然です。',
    rationale='1. 語尾の変換：丁寧語「～ます」を、ナルト特有の「～ぜ」「～ってばよ」に変更し、語調を強めます。\n2. 動作の言い換え：「議論を丁寧にまとめる」という堅い表現を、「きっちりまとめる」や「ビシッと終わらせる」といった、より直感的でエネルギッシュな言葉に置き換えます。\n3. 余計な一言の追加：指示通り『でもさ、』の後に、ナルトらしい「お腹が空いた」などの本音や、会議に対する苦手意識を付け加えます。',
    transformed='今日の会議の話し合いは、オレがビシッときっちりまとめてやるぜ！でもさ、小難しい話ばっかしてると、すぐにお腹が減ってきちゃうんだってばよ。'
)

### create an example data

In [16]:
easy_example = dspy.Example(
    polite_sentence="今日の会議では議論を丁寧にまとめます。",
    transformed="オレが今日の会議をビシッとまとめてやるってばよ！でもさ、終わったらラーメン一杯くらい付き合ってくれよな？",
    rationale="敬体をくだけた一人称『オレ』に置き換え、語尾へ『ってばよ』を追加し、『でもさ、』で余計なお願いをぶつけた。"
).with_inputs("polite_sentence")

### set evaluation criteria

In [17]:
# ナルト口調変換の質を自動評価するシグネチャを定義
class Assess(dspy.Signature):
    '''口調変換の観点をチェックし、改善ヒントを返す。'''
    assessment_transformed = dspy.InputField(desc="生成されたナルト口調の文")
    assessment_rationale = dspy.InputField(desc="生成時の推論メモ")
    assessment_input = dspy.InputField(desc="元の丁寧語文")
    assessment_question = dspy.InputField(desc="評価観点となる質問")
    assessment_answer = dspy.OutputField(desc="yes / no 判定")
    assessment_feedback = dspy.OutputField(desc="改善のヒント")

def _run_checks(gold, pred):
    polite = gold.get('polite_sentence', '')
    naruto_line = pred.get('transformed', '') if isinstance(pred, dict) else getattr(pred, 'transformed', '')
    rationale = pred.get('rationale', '') if isinstance(pred, dict) else getattr(pred, 'rationale', '')

    questions = [
        ("tone", "この文はナルトが使う一人称や『ってばよ』などの勢いある口調を含んでいますか？"),
        ("extra", "この文には『でもさ、』で始まる余計な一言が続いていますか？"),
        ("consistency", f"この文は元の丁寧語文『{polite}』の意味を保ちながら砕けた表現に変換していますか？"),
        ("rationale", "推論過程は採用した口調や余計な一言の意図を説明できていますか？")
    ]

    assessor = dspy.Predict(Assess)
    results = []
    feedback = []

    for key, question in questions:
        judgement = assessor(
            assessment_transformed=naruto_line,
            assessment_rationale=rationale,
            assessment_input=polite,
            assessment_question=question
        )
        answer = judgement.assessment_answer.lower() if judgement.assessment_answer else ''
        ok = 'yes' in answer
        results.append(ok)
        if (not ok) and judgement.assessment_feedback:
            feedback.append(judgement.assessment_feedback.strip())

    # 強制チェック: 『でもさ、』と『ってばよ』系が見つからない場合は強制減点
    if 'でもさ、' not in naruto_line:
        results.append(False)
        feedback.append("『でもさ、』で余計な一言を入れてください。")
    if ('ってばよ' not in naruto_line) and ('だってばよ' not in naruto_line):
        results.append(False)
        feedback.append("ナルトらしい語尾『ってばよ』を入れてください。")

    score = sum(results) / len(results) if results else 0.0
    return score, feedback

def metric(gold, pred, trace=None):
    score, _ = _run_checks(gold, pred)
    return round(score, 2)

def metric_with_feedback(example, prediction, trace=None, pred_name=None, pred_trace=None):
    score, feedback = _run_checks(example, prediction)
    message = os.linesep.join(feedback) if feedback else 'ナルト口調と余計な一言がうまく表現されています。'
    return dspy.Prediction(score=score, feedback=message)

# 使用例
metric(easy_example.inputs(), easy_example.labels())

0.75

### Optimization with COPRO

In [23]:
trainset = [easy_example]

In [22]:
from dspy.teleprompt import COPRO

prompt_optimizer = COPRO(metric=metric, verbose=True)
kwargs = dict(num_threads=64, display_progress=True, display_table=0)
# prompt_tuned = prompt_optimizer.compile(NarutoStyleChain(), trainset=trainset, eval_kwargs=kwargs)